In [17]:
import torch
import torch.nn as nn

In [40]:
def conv1d(input_signal, kernel):
    # Low pass filter
    input_len = len(input_signal)
    kernel_len = len(kernel)
    output_len = input_len - kernel_len + 1
    
    # Initialize the output signal
    output_signal = [0] * output_len
    
    # Perform convolution
    for i in range(output_len):
        output_signal[i] = sum(x * y for x, y in zip(input_signal[i:i+kernel_len], kernel))
    
    return output_signal

Output signal after 1D convolution: [2.0, 3.0, 4.0, 4.75, 4.75, 4.0, 3.0, 2.0]


In [42]:
def output_cnn(num_layers, input_vector, kernel):
    temp = input_vector
    for i in range(num_layers):
        hidden = conv1d(temp, kernel)
        temp = hidden
    output = hidden
    return output

In [49]:
# Example usage:
input_vector = [1, 2, 3, 4, 5, 5, 4, 3, 2, 1] 
kernel = [0.25, 0.5, 0.25]  # Example kernel
num_layers = 4

output_signal = output_cnn(num_layers, input_vector, kernel)
print("Output signal after 1D convolution:", output_signal)

Output signal after 1D convolution: [4.26953125, 4.26953125]


### Read Data

In [50]:
import pickle
import numpy as np
from datasets import load_dataset
import tqdm
import math

In [62]:
dataset = load_dataset("glue", "sst2")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [121]:
# Read pkl file 
with open('sst-train-768.pkl', 'rb') as f:
    vectors = pickle.load(f)

In [122]:
def average_distance_between_vectors(vectors):
    """
    Calculate the average distance between all pairs of vectors within a set.

    Parameters:
    vectors (numpy.ndarray): A 2D numpy array where each row represents a vector.

    Returns:
    float: The average distance between all pairs of vectors.
    """
    # Calculate pairwise distances using broadcasting
    pairwise_distances = np.linalg.norm(vectors[:, None] - vectors, axis=-1)
    
    # Exclude diagonal elements (distances between the same vectors)
    pairwise_distances = np.triu(pairwise_distances, k=1)
    
    # Calculate the average distance
    average_distance = np.mean(pairwise_distances)
    
    return average_distance

In [125]:
def get_avg_distance_based_on_layers(N, vectors, kernel):
    X = []
    for i in tqdm.tqdm(range(len(vectors))):
        # Sample usage
        input_sequence = output_cnn(N, vectors[i], kernel)
        X.append(input_sequence)

    X_np = np.array(X)
    std = np.mean(np.std(X_np, axis = 0) ** 2)
    return std

In [126]:
num_layers = [1, 2, 4, 8, 16, 32]
kernel = [0.33, 0.33, 0.33] 
for N in num_layers:
    dist = get_avg_distance_based_on_layers(N, vectors, kernel)
    print(N)
    print(dist)

100%|██████████| 67480/67480 [02:58<00:00, 378.58it/s]


1
3.6443162740708483e-28


100%|██████████| 67480/67480 [03:28<00:00, 323.18it/s]


2
2.0872327584549193e-28


100%|██████████| 67480/67480 [04:28<00:00, 251.08it/s]


...

8
1.1172032943710324e-28


 81%|████████  | 54494/67480 [08:13<01:56, 111.81it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 67480/67480 [1:05:24<00:00, 17.19it/s]    


16
6.111560807540266e-29


100%|██████████| 67480/67480 [17:25<00:00, 64.54it/s]


32
2.946367262636431e-29
